# Trabalho Final de Sudoku utilizando o LTNtorch

Este notebook resolve a **Questão 1**, **Questão 2** e **Questão 3** do trabalho: classificar se um tabuleiro de Sudoku fechado é válido com base nas regras do jogo, usando lógica de primeira ordem com a biblioteca `LTNtorch`, classificar um tabuleiro inicial / aberto (usar heurísticas) e indicar para um tabuleiro aberto quais heurísticas são mais recomendadas.

_____
**Integrantes**

* Alberth Viana de Lima
* Ana Júlia Pereira Corrêa
* Daniel Silveira Gonzalez
* Guilherme Sahdo Maciel
* Júlio Melo Campos
* Stepheson Custódio
_____

**Instalação do LTNtorch utilizando o pip**

In [11]:
!pip install git+https://github.com/tommasocarraro/LTNtorch

  Cloning https://github.com/tommasocarraro/LTNtorch to /tmp/pip-req-build-qqpsof6b
  Running command git clone --filter=blob:none --quiet https://github.com/tommasocarraro/LTNtorch /tmp/pip-req-build-qqpsof6b
  Resolved https://github.com/tommasocarraro/LTNtorch to commit d1bd98169cc2121f8cdd25ff99901e4589923c95
  Preparing metadata (setup.py) ... done


**Importação com o Google Drive**

In [12]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**Importação de Bibliotecas Necessárias**

In [13]:
import torch
import torch.nn as nn
import pandas as pd
import ltn

# **Questão 1**

Consiste em classificar um tabuleiro de Sudoku fechado (com todas as células preenchidas) como válido ou inválido.


1. Predicado "Different": Define a lógica do predicado Different. Este predicado, implementado como uma rede neural simples (DifferentModel), calcula a "diferença" entre dois números. Sua saída é próxima de 1 se os números são distintos e próxima de 0 se são iguais, permitindo que o LTN avalie a violação de restrições.

2. Carregar CSV: A função carregar_tabuleiro_csv lê o tabuleiro do Sudoku de um arquivo CSV, convertendo-o em um tensor PyTorch para processamento.

3. Geração dos Axiomas: A função gerar_axiomas constrói as restrições lógicas do Sudoku. Para cada linha, coluna e bloco do tabuleiro, ela cria axiomas utilizando o predicado Different para assegurar que todos os números já preenchidos sejam únicos dentro de cada uma dessas unidades. Apenas valores não-zero são considerados para a geração dos axiomas.

4. Classificador: A função classificar_tabuleiro avalia a validade do tabuleiro. Ela calcula a satisfação geral de todos os axiomas gerados, pegando o valor mínimo de verdade entre eles. Se esse valor mínimo de satisfação for igual ou superior a um limiar (0.95 por padrão), o tabuleiro é considerado válido; caso contrário, é inválido.

5. Execução Principal: O bloco if __name__ == "__main__": carrega um tabuleiro de exemplo e utiliza o classificador para determinar e imprimir se o tabuleiro é válido ou inválido de acordo com as regras do Sudoku.

In [14]:
# ---------- Predicado "Different" ----------
class DifferentModel(nn.Module):
    def forward(self, x, y):
        return 1.0 - torch.exp(-100 * torch.abs(x - y))

Different = ltn.Predicate(DifferentModel())

# ---------- Carregar CSV ----------
def carregar_tabuleiro_csv(path):
    df = pd.read_csv(path, header=None)
    return torch.tensor(df.values, dtype=torch.float32)

# ---------- Geração dos axiomas ----------
def gerar_axiomas(tabuleiro):
    n = tabuleiro.shape[0]
    axiomas = []

    # ---------- Linhas ----------
    for i in range(n):
        for j1 in range(n):
            for j2 in range(j1 + 1, n):
                v1_val = tabuleiro[i][j1].item()
                v2_val = tabuleiro[i][j2].item()
                if v1_val != 0 and v2_val != 0:
                    v1 = ltn.Constant(torch.tensor([v1_val]))
                    v2 = ltn.Constant(torch.tensor([v2_val]))
                    axiomas.append(Different(v1, v2))

    # ---------- Colunas ----------
    for j in range(n):
        for i1 in range(n):
            for i2 in range(i1 + 1, n):
                v1_val = tabuleiro[i1][j].item()
                v2_val = tabuleiro[i2][j].item()
                if v1_val != 0 and v2_val != 0:
                    v1 = ltn.Constant(torch.tensor([v1_val]))
                    v2 = ltn.Constant(torch.tensor([v2_val]))
                    axiomas.append(Different(v1, v2))

    # ---------- Blocos ----------
    b = int(n ** 0.5)
    for bi in range(0, n, b):
        for bj in range(0, n, b):
            celulas = []
            for i in range(bi, bi + b):
                for j in range(bj, bj + b):
                    val = tabuleiro[i][j].item()
                    if val != 0:
                        celulas.append(ltn.Constant(torch.tensor([val])))
            for k1 in range(len(celulas)):
                for k2 in range(k1 + 1, len(celulas)):
                    axiomas.append(Different(celulas[k1], celulas[k2]))

    return axiomas

# ---------- Classificador ----------
def classificar_tabuleiro(tabuleiro, limiar=0.95):
    axiomas = gerar_axiomas(tabuleiro)
    if not axiomas:
        return 1  # tabuleiro vazio é considerado válido
    verdades = torch.stack([ax.value for ax in axiomas]).squeeze()
    score = verdades.min()
    return 1 if score.item() >= limiar else 0

# ---------- Main ----------
if __name__ == "__main__":
    caminho = "/content/drive/MyDrive/trabalho-final-IA/tabuleiros-questao1/tabuleiro4x4-invalido.csv"  # coloque o nome/caminho do seu arquivo
    tabuleiro = carregar_tabuleiro_csv(caminho)
    res = classificar_tabuleiro(tabuleiro)
    print("1 - Tabuleiro VÁLIDO!" if res else "0 - Tabuleiro INVÁLIDO!")


0 - Tabuleiro INVÁLIDO!


# **Questão 2**

Este código utiliza Lógica Tensorial Neural (LTN) combinada com uma rede neural MLP para aprender a reconhecer jogadas válidas em um tabuleiro parcialmente preenchido de Sudoku. Também é capaz de sugerir as melhores jogadas com base na probabilidade aprendida. As etapas principais são:

1. Predicado "ValidMove": Define se uma jogada (posição e valor) é válida, utilizando uma MLP simples com 3 entradas: linha i, coluna j e valor v. A saída da rede é próxima de 1 para jogadas válidas e próxima de 0 para inválidas, permitindo que o predicado LTN (ValidMove) represente o conhecimento aprendido.

2. Carregamento do Tabuleiro: A função carregar_tabuleiro_csv lê um tabuleiro parcial de Sudoku (com zeros nas casas vazias) a partir de um arquivo .csv, convertendo os dados em um tensor PyTorch para o treinamento.

3. Geração de Dados de Treino: A função gerar_dados_treino seleciona posições aleatórias do tabuleiro e tenta preenchê-las com valores aleatórios. Para cada tentativa, determina se a jogada é válida com base nas regras do Sudoku, gerando dados rotulados para o treinamento supervisionado.

4. Treinamento do Modelo MLP: A função treinar_modelo utiliza os dados gerados para treinar a rede neural com BCELoss. Durante o treinamento, o predicado LTN é avaliado e ajustado via backpropagation usando o otimizador Adam.

5. Análise de Jogadas por Posição: A função sugerir_por_posicao_completa analisa todas as posições vazias do tabuleiro e estima a probabilidade de validade de cada valor (1 a n). Imprime os resultados ordenados, marcando ✅ para válidas (probabilidade ≥ 0.5) e ❌ para inválidas.

6. Simulação de Dois Movimentos: A função prever_dois_movimentos busca pares de jogadas com alta probabilidade (≥ 0.7), que não entrem em conflito com o estado atual do tabuleiro. Garante que ambas as jogadas sejam válidas segundo a rede e as regras do Sudoku.

7. Execução Principal: O bloco if __name__ == "__main__": carrega o tabuleiro parcial do CSV, treina o modelo, realiza a análise completa por posição e simula sequências de dois movimentos prováveis.

In [17]:
import torch
import torch.nn as nn
import pandas as pd
import ltn
import random

# ---------- Modelo MLP treinável ----------
class ValidMoveModel(nn.Module):
    def __init__(self, n=4):
        super().__init__()
        self.mlp = nn.Sequential(
            nn.Linear(3, 16),
            nn.ReLU(),
            nn.Linear(16, 8),
            nn.ReLU(),
            nn.Linear(8, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.mlp(x)

# ---------- Predicado LTN ----------
model = ValidMoveModel()
ValidMove = ltn.Predicate(model)

# ---------- Carrega tabuleiro ----------
def carregar_tabuleiro_csv(path):
    df = pd.read_csv(path, header=None)
    return torch.tensor(df.values, dtype=torch.float32)

# ---------- Verifica se (i,j,v) é válida ----------
def jogada_valida(tabuleiro, i, j, v):
    n = tabuleiro.shape[0]
    if v in tabuleiro[i] or v in tabuleiro[:, j]:
        return False
    b = int(n ** 0.5)
    bi, bj = i - i % b, j - j % b
    bloco = tabuleiro[bi:bi+b, bj:bj+b].flatten()
    return v not in bloco

# ---------- Gera dados de treino ----------
def gerar_dados_treino(tabuleiro, amostras=3000):
    n = tabuleiro.shape[0]
    X = []
    Y = []
    for _ in range(amostras):
        i = random.randint(0, n-1)
        j = random.randint(0, n-1)
        v = random.randint(1, n)
        if tabuleiro[i][j] != 0:
            continue
        x = torch.tensor([i, j, v], dtype=torch.float32)
        y = 1.0 if jogada_valida(tabuleiro, i, j, v) else 0.0
        X.append(x)
        Y.append(torch.tensor([y]))
    return torch.stack(X), torch.stack(Y)

# ---------- Treina a MLP ----------
def treinar_modelo(tabuleiro):
    X, Y = gerar_dados_treino(tabuleiro)
    const_X = ltn.Variable("x", X)
    label_Y = ltn.Constant(Y)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

    for epoch in range(500):
        optimizer.zero_grad()
        predicoes = ValidMove(const_X)
        loss = nn.BCELoss()(predicoes.value, Y.squeeze())
        loss.backward()
        optimizer.step()
        if epoch % 20 == 0:
            print(f"Epoch {epoch}: Loss = {loss.item():.4f}")
    print("Treinamento concluído.")

# ---------- Sugestões por posição (válidas e inválidas) ----------
def sugerir_por_posicao_completa(tabuleiro):
    n = tabuleiro.shape[0]
    for i in range(n):
        for j in range(n):
            if tabuleiro[i][j] != 0:
                continue
            print(f"\nPosição vazia ({i}, {j}):")
            jogadas = []
            for v in range(1, n+1):
                entrada = torch.tensor([[i, j, v]], dtype=torch.float32)
                prob = ValidMove(ltn.Constant(entrada)).value.item()
                status = "✅ válida" if prob >= 0.5 else "❌ inválida"
                jogadas.append((v, prob, status))
            # ordena do maior pro menor
            jogadas.sort(key=lambda x: -x[1])
            for v, prob, status in jogadas:
                print(f"   - Valor {v}: {status} com {prob:.4f}")

# ---------- Prever dois movimentos ----------
def prever_dois_movimentos(tabuleiro):
    n = tabuleiro.shape[0]
    resultados = []
    for i1 in range(n):
        for j1 in range(n):
            if tabuleiro[i1][j1] != 0:
                continue
            for v1 in range(1, n+1):
                entrada1 = torch.tensor([[i1, j1, v1]], dtype=torch.float32)
                prob1 = ValidMove(ltn.Constant(entrada1)).value.item()
                if prob1 < 0.7 or not jogada_valida(tabuleiro, i1, j1, v1):
                    continue
                novo_tabuleiro = tabuleiro.clone()
                novo_tabuleiro[i1][j1] = v1
                for i2 in range(n):
                    for j2 in range(n):
                        if novo_tabuleiro[i2][j2] != 0:
                            continue
                        for v2 in range(1, n+1):
                            entrada2 = torch.tensor([[i2, j2, v2]], dtype=torch.float32)
                            prob2 = ValidMove(ltn.Constant(entrada2)).value.item()
                            if prob2 >= 0.7 and jogada_valida(novo_tabuleiro, i2, j2, v2):
                                resultados.append(((i1,j1,v1,prob1), (i2,j2,v2,prob2)))
    return resultados


# ---------- Main ----------
if __name__ == "__main__":
    caminho = "/content/drive/MyDrive/trabalho-final-IA/tabuleiros-questao2/tabuleiro4x4-parcial.csv"
    tabuleiro = carregar_tabuleiro_csv(caminho)

    print("Treinando rede para prever jogadas válidas...\n")
    treinar_modelo(tabuleiro)

    print("\nAnálise completa por posição (válidas e inválidas):")
    sugerir_por_posicao_completa(tabuleiro)

    print("\nSimulando dois movimentos com alta probabilidade...\n")
    resultados = prever_dois_movimentos(tabuleiro)
    for (i1, j1, v1, p1), (i2, j2, v2, p2) in resultados:
        print(f"1º movimento ({i1},{j1})={v1} ({p1:.2f}) → 2º movimento ({i2},{j2})={v2} ({p2:.2f})")


Treinando rede para prever jogadas válidas...

Epoch 0: Loss = 0.6402
Epoch 20: Loss = 0.5807
Epoch 40: Loss = 0.4604
Epoch 60: Loss = 0.3476
Epoch 80: Loss = 0.2601
Epoch 100: Loss = 0.1956
Epoch 120: Loss = 0.1477
Epoch 140: Loss = 0.0900
Epoch 160: Loss = 0.0566
Epoch 180: Loss = 0.0327
Epoch 200: Loss = 0.0190
Epoch 220: Loss = 0.0113
Epoch 240: Loss = 0.0074
Epoch 260: Loss = 0.0053
Epoch 280: Loss = 0.0040
Epoch 300: Loss = 0.0032
Epoch 320: Loss = 0.0026
Epoch 340: Loss = 0.0022
Epoch 360: Loss = 0.0018
Epoch 380: Loss = 0.0016
Epoch 400: Loss = 0.0014
Epoch 420: Loss = 0.0012
Epoch 440: Loss = 0.0011
Epoch 460: Loss = 0.0010
Epoch 480: Loss = 0.0009
Treinamento concluído.

Análise completa por posição (válidas e inválidas):

Posição vazia (0, 1):
   - Valor 4: ✅ válida com 0.9999
   - Valor 3: ❌ inválida com 0.0057
   - Valor 1: ❌ inválida com 0.0000
   - Valor 2: ❌ inválida com 0.0000

Posição vazia (0, 2):
   - Valor 1: ✅ válida com 0.9979
   - Valor 4: ✅ válida com 0.9960
  

# **Questão 3**

Consiste em aplicar heurísticas para analisar um tabuleiro de Sudoku aberto, gerar cláusulas lógicas correspondentes, resolver com SAT-solver e discutir a possibilidade de resolução com LTN (Logic Tensor Networks).

1. Recomendação de Heurísticas: A função recomendar_heuristica analisa todas as células vazias do tabuleiro e calcula, para cada uma, quantas opções válidas ela possui. A partir dessa análise, recomenda o uso de heurísticas como MRV (Minimum Remaining Values) e LCV (Least Constraining Value), com base na proporção de células que têm 1 ou 2 possibilidades. Essa estratégia ajuda a priorizar células mais restritas e guiar a escolha dos valores que minimizam o impacto nas opções futuras.

2. Conversão para SAT (CNF): A função sudoku_to_cnf converte as regras do Sudoku (linhas, colunas, blocos e células preenchidas) em cláusulas CNF (forma normal conjuntiva), seguindo a codificação padrão para problemas SAT. Cada célula (i,j) é representada como uma variável booleana que indica se ela contém o valor v.

3. Resolução com SAT-Solver: A função resolver_sudoku_sat utiliza o solver Glucose3 da biblioteca PySAT para resolver o conjunto de cláusulas CNF. Se uma solução for encontrada, ela é reconstruída no formato de tabuleiro e impressa. Caso contrário, o sistema informa que o problema é insatisfatível.

4. Discussão sobre LTN: O código final inclui uma explicação teórica sobre como o Sudoku poderia ser resolvido com LTN. A ideia é representar os predicados e regras como axiomas lógicos, permitindo que a rede seja treinada para maximizar o grau de verdade dessas regras. Isso abre espaço para lidar com incertezas, dados incompletos e integrar aprendizado com lógica simbólica.

5. Execução Principal: O bloco if __name__ == "__main__" carrega o tabuleiro de um arquivo .csv, realiza a análise heurística, resolve o Sudoku com SAT e apresenta as considerações finais sobre o uso de LTN como alternativa de raciocínio aproximado e simbólico.

**Instalação do SAT Solver utilizando o pip**

In [18]:
!pip install python-sat[pblib,aiger]


In [19]:
import torch
import pandas as pd
import math
from pysat.formula import CNF
from pysat.solvers import Glucose3

# === Parte 1: Heurística ===

def carregar_tabuleiro_csv(path):
    df = pd.read_csv(path, header=None)
    return torch.tensor(df.values, dtype=torch.int32)

def jogada_valida(tabuleiro, i, j, v):
    n = tabuleiro.shape[0]
    if v in tabuleiro[i, :] or v in tabuleiro[:, j]:
        return False
    b = int(math.sqrt(n))
    bi, bj = i - i % b, j - j % b
    bloco = tabuleiro[bi:bi + b, bj:bj + b].flatten()
    return v not in bloco

def analisar_opcoes_celulas(tabuleiro):
    n = tabuleiro.shape[0]
    celulas_vazias_com_opcoes = []

    for i in range(n):
        for j in range(n):
            if tabuleiro[i, j] == 0:
                opcoes_validas = []
                for v in range(1, n + 1):
                    if jogada_valida(tabuleiro, i, j, v):
                        opcoes_validas.append(v)
                celulas_vazias_com_opcoes.append(((i, j), len(opcoes_validas)))

    celulas_vazias_com_opcoes.sort(key=lambda x: x[1])
    return celulas_vazias_com_opcoes

def recomendar_heuristica(tabuleiro):
    n = tabuleiro.shape[0]
    celulas_info = analisar_opcoes_celulas(tabuleiro)

    if not celulas_info:
        print("O tabuleiro está completo. Não há jogadas a serem feitas.")
        return

    num_celulas_vazias = len(celulas_info)

    # Contar quantas células têm poucas opções
    cont_poucas_opcoes = 0
    for _, num_opcoes in celulas_info:
        if num_opcoes <= 2: # Consideramos "poucas opções" se forem 1 ou 2
            cont_poucas_opcoes += 1

    percentual_poucas_opcoes = (cont_poucas_opcoes / num_celulas_vazias) * 100

    print(f"Análise do Tabuleiro ({n}x{n}):")
    print(f"Total de células vazias: {num_celulas_vazias}")
    print(f"Células vazias com 1 ou 2 opções: {cont_poucas_opcoes} ({percentual_poucas_opcoes:.2f}%)")

    print("\nDetalhes das células com menos opções:")
    # Imprime as 5 células com menos opções para dar uma ideia
    for k, ((r, c), num_ops) in enumerate(celulas_info[:min(5, num_celulas_vazias)]):
        print(f"  - Posição ({r},{c}): {num_ops} opções.")

    print("\n--- Recomendação de Heurística ---")

    if percentual_poucas_opcoes >= 50:
        print("Heurística Recomendada: MRV (Minimum Remaining Values)")
        print("Priorize preencher as células que possuem o menor número de opções válidas.")
        print("Isso ajuda a identificar 'gargalos' e a reduzir o espaço de busca mais rapidamente.")
        if cont_poucas_opcoes > 0 and celulas_info[0][1] == 1:
            print("Existem células com apenas 1 opção, o que indica um 'número escondido único' ou 'célula única'.")
    elif percentual_poucas_opcoes > 10:
        print("Heurística Principal: MRV (Minimum Remaining Values)")
        print("Ainda é recomendado focar nas células com menos opções, mas o tabuleiro pode ter mais flexibilidade.")
        print("Pode-se considerar a LCV (Least Constraining Value) ao escolher o valor para essas células.")
    else:
        print("Heurística Geral: MRV (Minimum Remaining Values) é sempre um bom ponto de partida.")
        print("No entanto, como muitas células têm múltiplas opções, o tabuleiro pode estar em um estágio inicial ou ter várias soluções possíveis.")
        print("Ao escolher o valor, a heurística LCV pode ser útil: escolha o valor que menos restringe as opções das células vizinhas.")

    print("\nLembre-se: Para resolver o Sudoku completamente, geralmente combinamos MRV (para escolher a célula) com LCV (para escolher o valor).")

# === Parte 2: SAT Solver ===

def var(i, j, v, n):
    return i * n * n + j * n + v

def sudoku_to_cnf(tabuleiro):
    n = tabuleiro.shape[0]
    cnf = CNF()

    # Regra 1: cada célula deve ter um valor
    for i in range(n):
        for j in range(n):
            cnf.append([var(i, j, v, n) for v in range(1, n + 1)])

    # Regra 2: cada célula pode ter no máximo um valor
    for i in range(n):
        for j in range(n):
            for v1 in range(1, n):
                for v2 in range(v1 + 1, n + 1):
                    cnf.append([-var(i, j, v1, n), -var(i, j, v2, n)])

    # Regra 3: linha
    for i in range(n):
        for v in range(1, n + 1):
            for j1 in range(n):
                for j2 in range(j1 + 1, n):
                    cnf.append([-var(i, j1, v, n), -var(i, j2, v, n)])

    # Regra 4: coluna
    for j in range(n):
        for v in range(1, n + 1):
            for i1 in range(n):
                for i2 in range(i1 + 1, n):
                    cnf.append([-var(i1, j, v, n), -var(i2, j, v, n)])

    # Regra 5: bloco
    b = int(math.sqrt(n))
    for bi in range(0, n, b):
        for bj in range(0, n, b):
            for v in range(1, n + 1):
                cells = []
                for i in range(bi, bi + b):
                    for j in range(bj, bj + b):
                        cells.append((i, j))
                for idx1 in range(len(cells)):
                    for idx2 in range(idx1 + 1, len(cells)):
                        i1, j1 = cells[idx1]
                        i2, j2 = cells[idx2]
                        cnf.append([-var(i1, j1, v, n), -var(i2, j2, v, n)])

    # Regra 6: células preenchidas
    for i in range(n):
        for j in range(n):
            v = int(tabuleiro[i, j].item())
            if v != 0:
                cnf.append([var(i, j, v, n)])

    return cnf

def resolver_sudoku_sat(tabuleiro):
    cnf = sudoku_to_cnf(tabuleiro)
    solver = Glucose3()
    solver.append_formula(cnf)

    if solver.solve():
        modelo = solver.get_model()
        n = tabuleiro.shape[0]
        sol = torch.zeros_like(tabuleiro)

        for lit in modelo:
            if lit > 0:
                v = lit - 1
                i = v // (n * n)
                j = (v % (n * n)) // n
                valor = (v % n) + 1
                sol[i, j] = valor
        print("\nSolução encontrada com SAT Solver:")
        print(sol.int().numpy())
    else:
        print("\nO tabuleiro não tem solução (insatisfatível).")

# === Main ===

if __name__ == "__main__":
    caminho = "/content/drive/MyDrive/trabalho-final-IA/tabuleiros-questao1/tabuleiro4x4-vazio.csv"
    tabuleiro = carregar_tabuleiro_csv(caminho)

    print("Análise heurística:")
    recomendar_heuristica(tabuleiro)

    print("\nTentando resolver com SAT solver:")
    resolver_sudoku_sat(tabuleiro)

    print("\nConsiderações sobre o uso de LTN:")

    print("""
    Sim, seria possível resolver o Sudoku utilizando LTN (Logic Tensor Networks).
    A LTN permite integrar lógica simbólica (como as regras do Sudoku) com aprendizado baseado em tensores, possibilitando que as restrições sejam tratadas como fórmulas lógicas com graus de verdade (fuzzy logic).

    Nesse contexto, o Sudoku pode ser formulado com predicados como:
    - cell(i, j, v): verdadeiro se a célula (i, j) contém o valor v;
    - diff(A, B): verdadeiro se os valores A e B são diferentes (usado para garantir valores únicos em linhas, colunas e blocos).

    As regras tradicionais do Sudoku são inseridas como axiomas lógicos no sistema LTN, e a rede é treinada para satisfazê-las com a maior verdade possível.

    Isso é útil especialmente em casos com ruído ou tabuleiros incompletos, onde uma solução aproximada ainda é aceitável.
    Além disso, a LTN pode combinar aprendizado supervisionado (usando exemplos de Sudokus resolvidos) com raciocínio lógico simbólico, o que é vantajoso em cenários com poucos dados mas muitas regras explícitas.
    """)


Análise heurística:
Análise do Tabuleiro (4x4):
Total de células vazias: 16
Células vazias com 1 ou 2 opções: 0 (0.00%)

Detalhes das células com menos opções:
  - Posição (0,0): 4 opções.
  - Posição (0,1): 4 opções.
  - Posição (0,2): 4 opções.
  - Posição (0,3): 4 opções.
  - Posição (1,0): 4 opções.

--- Recomendação de Heurística ---
Heurística Geral: MRV (Minimum Remaining Values) é sempre um bom ponto de partida.
No entanto, como muitas células têm múltiplas opções, o tabuleiro pode estar em um estágio inicial ou ter várias soluções possíveis.
Ao escolher o valor, a heurística LCV pode ser útil: escolha o valor que menos restringe as opções das células vizinhas.

Lembre-se: Para resolver o Sudoku completamente, geralmente combinamos MRV (para escolher a célula) com LCV (para escolher o valor).

Tentando resolver com SAT solver:

Solução encontrada com SAT Solver:
[[1 2 3 4]
 [3 4 1 2]
 [2 1 4 3]
 [4 3 2 1]]

Considerações sobre o uso de LTN:

    Sim, seria possível resolver o S

# **Questão (todos os cenários) - sem treinamento**

Esta outra versão do trabalho final foca na projeção e implementação de uma solução para o Sudoku utilizando LTNTorch, visando "aprender os conceitos" conforme as questões propostas. Para a:

* Questão 1, que classifica tabuleiros fechados, utilizou-se o framework Logic Tensor Networks (LTN) com um predicado Different para verificar a satisfação das restrições lógicas do Sudoku.
* Já as Questões 2 e 3, que abordam tabuleiros abertos, foram implementadas com base em NumPy para classificar enigmas como "sem solução" ou "solução possível" e para indicar as heurísticas mais recomendadas, como MRV.

In [20]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import ltn
import sys
import os
import copy # Para copy.deepcopy

# --- Constantes Globais (serão definidas dinamicamente por set_board_size) ---
BOARD_SIZE = None
BLOCK_SIZE = None

# --- Predicado LTN "Different" ---
class DifferentModel(nn.Module):
    def forward(self, x, y):
        return 1.0 - torch.exp(-100 * torch.abs(x - y))

Different = ltn.Predicate(DifferentModel())

**Funções Auxiliares Comuns**

In [21]:
# --- Funções Auxiliares Comuns ---
def set_board_size(board_np):
    global BOARD_SIZE, BLOCK_SIZE
    BOARD_SIZE = board_np.shape[0]
    BLOCK_SIZE = int(np.sqrt(BOARD_SIZE))
    if BLOCK_SIZE * BLOCK_SIZE != BOARD_SIZE:
        print(f"Erro: O tamanho do tabuleiro {BOARD_SIZE} não é um quadrado perfeito (necessário para Sudoku).")
        return False
    return True

def carregar_tabuleiro_csv(path):
    print(f"\nLendo o tabuleiro do arquivo '{path}'...")
    try:
        df = pd.read_csv(path, header=None)
        board_np = df.values.astype(int)
        board_tensor = torch.tensor(board_np, dtype=torch.float32)

        if not set_board_size(board_np):
            return None, None

        if board_np.shape[0] != board_np.shape[1]:
            print(f"ERRO: O tabuleiro no arquivo '{path}' tem dimensões não quadradas {board_np.shape}.")
            return None, None

        if np.any(board_np < 0) or np.any(board_np > BOARD_SIZE):
            print(f"ERRO: O tabuleiro contém números fora do intervalo válido [0, {BOARD_SIZE}].")
            return None, None

        print("Tabuleiro Lido:\n", board_np)
        return board_np, board_tensor

    except FileNotFoundError:
        print(f"ERRO: Arquivo '{path}' não encontrado.")
        return None, None
    except ValueError:
        print(f"ERRO: O arquivo '{path}' não parece ser um CSV válido para o tabuleiro (contém não-números ou formato incorreto).")
        return None, None
    except Exception as e:
        print(f"ERRO inesperado ao carregar o tabuleiro: {e}")
        return None, None

def get_empty_cells(board_np):
    cells = []
    for r in range(BOARD_SIZE):
        for c in range(BOARD_SIZE):
            if board_np[r, c] == 0:
                cells.append((r, c))
    return cells

def violates_constraint(board_np, move):
    row, col, digit = move

    if not (1 <= digit <= BOARD_SIZE):
        return True

    if digit in board_np[row, :]:
        return True
    if digit in board_np[:, col]:
        return True
    start_row, start_col = (row // BLOCK_SIZE) * BLOCK_SIZE, (col // BLOCK_SIZE) * BLOCK_SIZE
    if digit in board_np[start_row:start_row + BLOCK_SIZE, start_col:start_col + BLOCK_SIZE]:
        return True
    return False

**Lógica da Questão 1 - Classificar Tabuleiro Fechado (usando LTN)**

Nesta questão, o objetivo foi classificar um tabuleiro de Sudoku que está completamente preenchido (fechado), independentemente de seu tamanho ser 4×4 ou 9×9. A solução implementada utiliza o framework Logic Tensor Networks (LTN) para verificar a validade do tabuleiro. Para isso, foi definido um predicado Different em LTN, que mede o quão "diferentes" são dois valores.

Em seguida, foram gerados axiomas lógicos para as restrições do Sudoku (linhas, colunas e blocos), utilizando esse predicado para garantir que todos os números em cada unidade (linha, coluna ou bloco) sejam únicos. O sistema calcula a satisfação geral desses axiomas. Se a satisfação estiver acima de um limiar predefinido (0.99), o tabuleiro é classificado como 1 (corretamente preenchido); caso contrário, é classificado como 0 (violou alguma restrição).

In [22]:
# --- Lógica da Questão 1: Classificar Tabuleiro Fechado (usando LTN Different) ---
def gerar_axiomas_fechado(tabuleiro_tensor):
    n = tabuleiro_tensor.shape[0]
    b = int(n ** 0.5)
    axiomas = []

    for i in range(n):
        filled_values_in_row = []
        for j in range(n):
            val = tabuleiro_tensor[i, j].item()
            if val != 0:
                filled_values_in_row.append(ltn.Constant(torch.tensor([val])))

        for k1 in range(len(filled_values_in_row)):
            for k2 in range(k1 + 1, len(filled_values_in_row)):
                axiomas.append(Different(filled_values_in_row[k1], filled_values_in_row[k2]))

    for j in range(n):
        filled_values_in_col = []
        for i in range(n):
            val = tabuleiro_tensor[i, j].item()
            if val != 0:
                filled_values_in_col.append(ltn.Constant(torch.tensor([val])))

        for k1 in range(len(filled_values_in_col)):
            for k2 in range(k1 + 1, len(filled_values_in_col)):
                axiomas.append(Different(filled_values_in_col[k1], filled_values_in_col[k2]))

    for bi in range(0, n, b):
        for bj in range(0, n, b):
            filled_values_in_block = []
            for i in range(bi, bi + b):
                for j in range(bj, bj + b):
                    val = tabuleiro_tensor[i, j].item()
                    if val != 0:
                        filled_values_in_block.append(ltn.Constant(torch.tensor([val])))

            for k1 in range(len(filled_values_in_block)):
                for k2 in range(k1 + 1, len(filled_values_in_block)):
                    axiomas.append(Different(filled_values_in_block[k1], filled_values_in_block[k2]))

    return axiomas

def classificar_cenario1_tabuleiro_fechado(tabuleiro_np, tabuleiro_tensor, limiar=0.99):
    print("\n--- CENÁRIO 1: CLASSIFICAR TABULEIRO FECHADO (Questão 1) ---")

    if np.any(tabuleiro_np == 0):
        print("Aviso: O tabuleiro contém células vazias (0). A classificação verificará apenas a validade dos números preenchidos.")

    axiomas = gerar_axiomas_fechado(tabuleiro_tensor)

    if not axiomas:
        score = torch.tensor(1.0)
    else:
        verdades = torch.stack([ax.value for ax in axiomas]).squeeze()
        score = verdades.min()

    print(f"Satisfação geral dos axiomas LTN: {score.item():.4f}")

    if score.item() >= limiar:
        print(f"Classificação para CENÁRIO 1: 1 - ✅ Tabuleiro VÁLIDO (Satisfação >= {limiar})!")
        return 1
    else:
        print(f"Classificação para CENÁRIO 1: 0 - ❌ Tabuleiro INVÁLIDO (Satisfação < {limiar})!")
        return 0

**Lógica da Questão 2 - Classificar Tabuleiro Aberto e Cenários (usando Numpy)**

Para esta questão, o foco foi analisar um tabuleiro de Sudoku que possui células vazias (aberto). A classificação principal determina se o enigma:


1) Não possui solução: Ocorre se houver um numeral válido (1 a 4 para 4×4 ou 1 a 9 para 9×9) que não pode ser colocado em nenhuma célula vazia restante sem violar as restrições do Sudoku.


2) Possui solução possível: Classificação aplicada caso a situação anterior não se verifique.
___


Além da classificação, o sistema indica os numerais que têm maior probabilidade de levar a um impasse (situação sem solução) ou de manter uma solução possível, analisando:

a. Movimentos em um passo: Simula-se a colocação de cada numeral em cada célula vazia, avaliando o estado do tabuleiro resultante (se leva a "sem solução" ou "solução possível").

b. Movimentos em dois passos: A análise é estendida para sequências de dois movimentos, identificando padrões que resultam em impasses ou mantêm a viabilidade da solução.

In [23]:
# --- Lógica da Questão 2: Classificar Tabuleiro Aberto e Cenários (usando Numpy) ---
def check_sem_solucao(board_np):
    empty_cells = get_empty_cells(board_np)
    if not empty_cells:
        return False

    for digit_to_check in range(1, BOARD_SIZE + 1):
        can_place_digit = False
        for cell in empty_cells:
            move = (cell[0], cell[1], digit_to_check)
            if not violates_constraint(board_np, move):
                can_place_digit = True
                break

        if not can_place_digit:
            print(f"DIAGNÓSTICO: O dígito '{digit_to_check}' não pode ser colocado em nenhuma célula vazia restante.")
            return True
    return False

def evaluate_one_move(board_np):
    empty_cells = get_empty_cells(board_np)
    moves_leading_to_sem_solucao = []
    moves_maintaining_solucao_possivel = []

    for row, col in empty_cells:
        for digit in range(1, BOARD_SIZE + 1):
            move = (row, col, digit)

            if violates_constraint(board_np, move):
                continue

            copied_board = np.copy(board_np)
            copied_board[row, col] = digit

            if check_sem_solucao(copied_board):
                moves_leading_to_sem_solucao.append(move)
            else:
                moves_maintaining_solucao_possivel.append(move)
    return moves_leading_to_sem_solucao, moves_maintaining_solucao_possivel

def evaluate_two_moves(board_np):
    empty_cells = get_empty_cells(board_np)
    two_moves_leading_to_sem_solucao = []
    two_moves_maintaining_solucao_possivel = []

    for r1, c1 in empty_cells:
        for d1 in range(1, BOARD_SIZE + 1):
            first_move = (r1, c1, d1)

            if violates_constraint(board_np, first_move):
                continue

            board_after_first_move = copy.deepcopy(board_np)
            board_after_first_move[r1, c1] = d1

            if check_sem_solucao(board_after_first_move):
                two_moves_leading_to_sem_solucao.append((first_move, "DIAG: 1º passo leva a Sem Solução"))
                continue

            empty_cells_after_first_move = get_empty_cells(board_after_first_move)

            if not empty_cells_after_first_move:
                continue

            for r2, c2 in empty_cells_after_first_move:
                if (r1, c1) == (r2, c2): # Prevents putting a second digit in the same spot just filled by the first move
                    continue

                for d2 in range(1, BOARD_SIZE + 1):
                    second_move = (r2, c2, d2)

                    if violates_constraint(board_after_first_move, second_move):
                        continue

                    board_after_two_moves = copy.deepcopy(board_after_first_move)
                    board_after_two_moves[r2, c2] = d2

                    if check_sem_solucao(board_after_two_moves):
                        two_moves_leading_to_sem_solucao.append((first_move, second_move))
                    else:
                        two_moves_maintaining_solucao_possivel.append((first_move, second_move))

    return two_moves_leading_to_sem_solucao, two_moves_maintaining_solucao_possivel

def classificar_cenario2_tabuleiro_aberto(board_np):
    print("\n--- CENÁRIO 2: CLASSIFICAR TABULEIRO ABERTO (Questão 2) ---")

    empty_cells = get_empty_cells(board_np)
    if not empty_cells:
        print("O tabuleiro está completo. Não há células vazias para analisar.")
        print("Classificação para CENÁRIO 2: O tabuleiro está preenchido, não se aplica 'aberto'.")
        return 0

    is_sem_solucao = check_sem_solucao(board_np)

    if is_sem_solucao:
        print("Classificação para CENÁRIO 2: 1) Sem Solução")
    else:
        print("Classificação para CENÁRIO 2: 2) Solução Possível")

        print("\n--- CENÁRIO 2a: AVALIAÇÃO DE MOVIMENTOS EM UM PASSO ---")
        moves_leading_to_sem_solucao_one_step, moves_maintaining_solucao_possivel_one_step = evaluate_one_move(board_np)
        print(f"Movimentos que levam a 'Sem Solução': {moves_leading_to_sem_solucao_one_step}")
        print(f"Movimentos que mantêm 'Solução Possível': {moves_maintaining_solucao_possivel_one_step}")

        if moves_leading_to_sem_solucao_one_step:
            digits_causing_impasse = {}
            for r, c, d in moves_leading_to_sem_solucao_one_step:
                digits_causing_impasse[d] = digits_causing_impasse.get(d, 0) + 1
            sorted_digits_impasse = sorted(digits_causing_impasse.items(), key=lambda item: item[1], reverse=True)
            print(f"Numerais com maior probabilidade de levar a 'Sem Solução' (em 1 movimento): {sorted_digits_impasse}")
        else:
            print("Nenhum movimento em um passo leva a 'Sem Solução'.")

        print("\n--- CENÁRIO 2b: AVALIAÇÃO DE SEQUÊNCIAS DE DOIS MOVIMENTOS ---")
        two_moves_leading_to_sem_solucao, two_moves_maintaining_solucao_possivel = evaluate_two_moves(board_np)
        print(f"Sequências de dois movimentos que levam a 'Sem Solução': {two_moves_leading_to_sem_solucao}")
        print(f"Sequências de dois movimentos que mantêm 'Solução Possível': {two_moves_maintaining_solucao_possivel}")

        if two_moves_leading_to_sem_solucao:
            digits_causing_impasse_two_steps = {}
            for move1, move2 in two_moves_leading_to_sem_solucao:
                digits_causing_impasse_two_steps[move1[2]] = digits_causing_impasse_two_steps.get(move1[2], 0) + 1
            sorted_digits_impasse_two_steps = sorted(digits_causing_impasse_two_steps.items(), key=lambda item: item[1], reverse=True)
            print(f"Numerais com maior probabilidade de levar a 'Sem Solução' (em 2 movimentos, considerando o 1º dígito): {sorted_digits_impasse_two_steps}")
        else:
            print("Nenhuma sequência de dois movimentos leva a 'Sem Solução'.")

**Lógica da Questão 3 - Indicar Heurísticas Mais Recomendadas (usando Numpy)**

Nesta questão, o objetivo foi indicar heurísticas relevantes para a resolução de um tabuleiro de Sudoku aberto. Foram implementadas e comparadas duas heurísticas clássicas de problemas de satisfação de restrições:

* Heurística MRV (Minimum Remaining Values): Identifica as células vazias que possuem o menor número de opções válidas para preenchimento. Priorizar essas células pode ajudar a detectar impasses mais cedo.

* Heurística do Dígito Mais Restrito: Foca nos numerais que podem ser colocados no menor número de posições válidas no tabuleiro. Alocar esses dígitos pode evitar problemas futuros.

In [24]:
# --- Lógica da Questão 3: Indicar Heurísticas Mais Recomendadas (usando Numpy) ---
def initialize_binary_variables(board_np):
    binary_vars = np.zeros((BOARD_SIZE, BOARD_SIZE, BOARD_SIZE + 1), dtype=int)
    for i in range(BOARD_SIZE):
        for j in range(BOARD_SIZE):
            num = board_np[i, j]
            if num != 0:
                if 1 <= num <= BOARD_SIZE:
                    binary_vars[i, j, num] = 1
    return binary_vars

def is_valid_for_heuristic(binary_vars_3d, row, col, num):
    if np.sum(binary_vars_3d[row, :, num]) > 0:
        return False
    if np.sum(binary_vars_3d[:, col, num]) > 0:
        return False
    block_row_start = (row // BLOCK_SIZE) * BLOCK_SIZE
    block_col_start = (col // BLOCK_SIZE) * BLOCK_SIZE
    if np.sum(binary_vars_3d[block_row_start:block_row_start + BLOCK_SIZE,
                             block_col_start:block_col_start + BLOCK_SIZE, num]) > 0:
        return False
    return True

def heuristic_mrv(board_np_original):
    binary_vars = initialize_binary_variables(board_np_original)
    empty_cells = get_empty_cells(board_np_original)

    cell_options = []
    for r, c in empty_cells:
        options = []
        for num in range(1, BOARD_SIZE + 1):
            if is_valid_for_heuristic(binary_vars, r, c, num):
                options.append(num)
        cell_options.append(((r, c), options))

    cell_options.sort(key=lambda x: len(x[1]))
    return cell_options

def heuristic_most_constrained_digit(board_np_original):
    binary_vars = initialize_binary_variables(board_np_original)
    empty_cells = get_empty_cells(board_np_original)

    digit_spaces = {}
    for num in range(1, BOARD_SIZE + 1):
        count = 0
        for r, c in empty_cells:
            if is_valid_for_heuristic(binary_vars, r, c, num):
                count += 1
        digit_spaces[num] = count

    sorted_digits = sorted(digit_spaces.items(), key=lambda x: x[1])
    return sorted_digits

def classificar_cenario3_indicar_heuristicas(board_np):
    print("\n--- CENÁRIO 3: INDICAR HEURÍSTICAS MAIS RECOMENDADAS (Questão 3) ---")

    empty_cells = get_empty_cells(board_np)
    if not empty_cells:
        print("O tabuleiro está completo. Não há células vazias para aplicar heurísticas.")
        return

    print("\n--- Heurística MRV (Minimum Remaining Values) ---")
    mrv_results = heuristic_mrv(board_np)
    if mrv_results:
        print("Células com o menor número de opções válidas (mais restritas):")
        for (r, c), opts in mrv_results[:min(5, len(mrv_results))]:
            print(f"   Célula ({r},{c}) → {len(opts)} possibilidade(s): {opts}")
    else:
        print("Não há células vazias para aplicar MRV.")

    print("\n--- Heurística 'Dígito Mais Restrito' ---")
    most_constrained_digit_results = heuristic_most_constrained_digit(board_np)
    if most_constrained_digit_results:
        print("Dígitos com o menor número de posições válidas no tabuleiro (mais restritos):")
        for digit, count in most_constrained_digit_results[:min(4, len(most_constrained_digit_results))]:
            print(f"   Dígito {digit} → {count} posição(ões) possível(is)")
    else:
        print("Não há dígitos para analisar em células vazias.")

    print("\nConsiderações sobre as heurísticas:")
    print("A heurística MRV foca na célula mais difícil de preencher. Preenche-la primeiro pode expor um impasse cedo.")
    print("A heurística do Dígito Mais Restrito foca no dígito mais difícil de alocar. Alocá-lo pode evitar problemas futuros.")
    print("Ambas são úteis para guiar a busca em problemas de satisfação de restrições.")

    print("\n--- Pergunta Final: Resolução de Sudoku com LTN (Restrições + Heurísticas) ---")
    print("Sim, é possível resolver o Sudoku usando apenas LTN combinando restrições e heurísticas.")
    print("Para isso, o modelo 'DummyModel' precisaria ser substituído por uma Rede Neural real (ex: MLP).")
    print("Essa rede aprenderia a 'confiança' de cada número em cada célula.")
    print("Os axiomas LTN (como 'Different', e outros para 'HasNumber' e 'IsFilled' para células vazias)")
    print("formariam a função de perda. Otimizadores (SGD, Adam) ajustariam os pesos da rede para maximizar")
    print("a satisfação geral dos axiomas. As heurísticas seriam incorporadas como cláusulas LTN adicionais")
    print("para guiar o aprendizado da rede, priorizando certos tipos de atribuições.")
    print("Por exemplo, penalizando soluções onde uma célula tem muitas opções ou favorecendo a alocação de dígitos raros.")
    print("Após o treinamento, o tabuleiro seria resolvido selecionando os números com maior confiança pela rede.")

**Função Principal (Main) para Execução**

In [25]:
if __name__ == "__main__":
    # --- DEFINA A PASTA RAIZ DO SEU TRABALHO AQUI ---
    base_directory_path = "/content/drive/MyDrive/trabalho-final-IA/" # AJUSTE ESTE CAMINHO CONFORME SEU DRIVE

    if not os.path.isdir(base_directory_path):
        print(f"ERRO: O caminho base '{base_directory_path}' não é um diretório válido ou não foi montado corretamente.")
        print("Certifique-se de que o Google Drive está montado e o caminho está correto.")
        sys.exit(1)

    # Define as subpastas para as questões
    subfolders = {
        "questao1": os.path.join(base_directory_path, "tabuleiros-questao1"),
        "questao2": os.path.join(base_directory_path, "tabuleiros-questao2"),
        "questao3": os.path.join(base_directory_path, "tabuleiros-questao3") # Adicionada a pasta da Questão 3
    }

    # Processa tabuleiros da Questão 1
    print("\n==== Processando tabuleiros para CENÁRIO 1 (Questão 1) ====")
    if os.path.isdir(subfolders["questao1"]):
        csv_files_q1 = [f for f in os.listdir(subfolders["questao1"]) if f.endswith(".csv")]
        if not csv_files_q1:
            print(f"Nenhum arquivo CSV encontrado em '{subfolders['questao1']}'")
        for filename in csv_files_q1:
            file_path = os.path.join(subfolders["questao1"], filename)
            board_np, board_tensor = carregar_tabuleiro_csv(file_path)
            if board_np is not None and board_tensor is not None:
                print(f"\n==== ANÁLISE PARA O ARQUIVO: {filename} ====")
                classificar_cenario1_tabuleiro_fechado(board_np, board_tensor)
            else:
                print(f"\n==== Pulando arquivo: {filename} devido a erros de carregamento/validação ====")
    else:
        print(f"ERRO: Pasta '{subfolders['questao1']}' não encontrada. Pulando Questão 1.")


    # Processa tabuleiros da Questão 2 e 3
    print("\n==== Processando tabuleiros para CENÁRIO 2 e 3 (Questões 2 e 3) ====")
    # Lista de pastas a serem processadas para as Questões 2 e 3
    q2_q3_folders = [subfolders["questao2"], subfolders["questao3"]]

    for folder_path in q2_q3_folders:
        if os.path.isdir(folder_path):
            print(f"\n--- Processando tabuleiros da pasta: {os.path.basename(folder_path)} ---")
            csv_files_q2_q3 = [f for f in os.listdir(folder_path) if f.endswith(".csv")]
            if not csv_files_q2_q3:
                print(f"Nenhum arquivo CSV encontrado em '{folder_path}'")
            for filename in csv_files_q2_q3:
                file_path = os.path.join(folder_path, filename)
                board_np, board_tensor = carregar_tabuleiro_csv(file_path)
                if board_np is not None and board_tensor is not None:
                    print(f"\n==== ANÁLISE PARA O ARQUIVO: {filename} ====")
                    if np.any(board_np == 0): # Só executa Q2 e Q3 se o tabuleiro tiver células vazias
                        classificar_cenario2_tabuleiro_aberto(board_np)
                        classificar_cenario3_indicar_heuristicas(board_np)
                    else:
                        print("\n--- CENÁRIO 2 & 3: Não aplicável. O tabuleiro está completo. ---")
                else:
                    print(f"\n==== Pulando arquivo: {filename} devido a erros de carregamento/validação ====")
        else:
            print(f"ERRO: Pasta '{os.path.basename(folder_path)}' não encontrada. Pulando processamento desta pasta.")


==== Processando tabuleiros para CENÁRIO 1 (Questão 1) ====

Lendo o tabuleiro do arquivo '/content/drive/MyDrive/trabalho-final-IA/tabuleiros-questao1/tabuleiro4x4-valido.csv'...
Tabuleiro Lido:
 [[1 2 3 4]
 [3 4 1 2]
 [2 1 4 3]
 [4 3 2 1]]

==== ANÁLISE PARA O ARQUIVO: tabuleiro4x4-valido.csv ====

--- CENÁRIO 1: CLASSIFICAR TABULEIRO FECHADO (Questão 1) ---
Satisfação geral dos axiomas LTN: 1.0000
Classificação para CENÁRIO 1: 1 - ✅ Tabuleiro VÁLIDO (Satisfação >= 0.99)!

Lendo o tabuleiro do arquivo '/content/drive/MyDrive/trabalho-final-IA/tabuleiros-questao1/tabuleiro9x9-valido.csv'...
Tabuleiro Lido:
 [[5 3 4 6 7 8 9 1 2]
 [6 7 2 1 9 5 3 4 8]
 [1 9 8 3 4 2 5 6 7]
 [8 5 9 7 6 1 4 2 3]
 [4 2 6 8 5 3 7 9 1]
 [7 1 3 9 2 4 8 5 6]
 [9 6 1 5 3 7 2 8 4]
 [2 8 7 4 1 9 6 3 5]
 [3 4 5 2 8 6 1 7 9]]

==== ANÁLISE PARA O ARQUIVO: tabuleiro9x9-valido.csv ====

--- CENÁRIO 1: CLASSIFICAR TABULEIRO FECHADO (Questão 1) ---
Satisfação geral dos axiomas LTN: 1.0000
Classificação para CENÁRIO 1: 1